In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input,decode_predictions

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
WIDTH=299
HEIGHT=299
BATCH_SIZE=64
test_dir="/content/drive/My Drive/database_folder/car_theft/test"
train_dir="/content/drive/My Drive/database_folder/car_theft/train"
val_dir="/content/drive/My Drive/database_folder/car_theft/val"

important thing of data generator

here we are calling data_generator flow from directory 
in this mode string, path to the target directory. It should contain one subdirectory per class. 

each subdirectory will be treated as a different class

aasan sabdo me

in this call we do't need to provide y label,the name of the directory(folder/file) will be consider as y label for the data(img over here)

class mode : return augmunted image with 2-d one hot encoded label for deep learning

if class mode is none it will not label /classsify the augmunted image

In [6]:
# Data set generator with augemnataion from keras

#train data
print('Training data set')
train_generator=ImageDataGenerator(preprocessing_function=preprocess_input)
train_flow=train_generator.flow_from_directory(train_dir,
                                               target_size=(HEIGHT,WIDTH),
                                               batch_size=BATCH_SIZE)

Training data set
Found 4843 images belonging to 10 classes.


In [7]:
print('validation data set')
val_generator=ImageDataGenerator(preprocessing_function=preprocess_input)
val_flow=val_generator.flow_from_directory(val_dir,
                                               target_size=(HEIGHT,WIDTH),
                                               batch_size=BATCH_SIZE)

validation data set
Found 0 images belonging to 0 classes.


In [8]:
print('test data set')
test_generator=ImageDataGenerator(preprocessing_function=preprocess_input)
test_flow=test_generator.flow_from_directory(test_dir,
                                               target_size=(HEIGHT,WIDTH),
                                               batch_size=BATCH_SIZE)

test data set
Found 1196 images belonging to 10 classes.


optimization of cpu


something i understand only a bit

In [0]:
from keras.models import Sequential,Model,load_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger
from keras import optimizers,models
from keras.layers import Dense,Dropout,GlobalAveragePooling2D
from keras import applications
from keras import backend as K
import tensorflow as tf
import os


In [0]:
NUM_PARALLEL_EXEC_UNITS=8


In [0]:
#setting parameter for parallel processing
config=tf.ConfigProto(
    intra_op_parallelism_threads=NUM_PARALLEL_EXEC_UNITS,
    inter_op_parallelism_threads=1
)

session=tf.Session(config=config)
K.set_session(session)

In [0]:

#MKL and OpenMP
os.environ["OMP_NUM_THREADS"] = str(NUM_PARALLEL_EXEC_UNITS)
os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

hyperparameter  setting of model

In [0]:
#batch_size
#learning_rate
#gradient descent 
#optimizer for gradient descent :momentum /adagrad/rmsprop etc
#transverse learning -in this we use the weights of some similar model as our initial weights
        # this speed up our model training
        #here we use Imagenet weights
        #we can't be able to copy top layer weights as change in number of feature in initial dataset

#initializing top layer



building model


pay attention when did 2d image is get converting into 1d data vector/flatten

In [13]:
#initialize inception v3 with transfer learning

base_model=applications.InceptionV3(weights='imagenet',include_top=False,
                                    input_shape=(WIDTH,HEIGHT,3))












87916544/87910968 [==============================] - 7s 0us/step


In [14]:
x=base_model.output
#add a average pooling

x=GlobalAveragePooling2D()(x)
#add a dense layer

x=Dense(1024,activation='relu')(x)
#add dense layer with softmax for prediction
#this will be final prediction layer

predictions=Dense(len(train_flow.class_indices),activation='softmax')(x)

model=Model(inputs=base_model.input,output=predictions)

#first train only the top layer
#freez all inception layer
for layer in base_model.layers:
  layer.trainable=False

#compling model

model.compile(optimizer=optimizers.Adam(lr=.001),metrics=['accuracy','top_k_categorical_accuracy'],loss='categorical_crossentropy')
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


In [0]:
# to create picture of model

from tensorflow.keras.utils import plot_model
plot_model(model, to_file='/content/drive/My Drive/database_folder/car_theft/models/model.png')

In [0]:
# Training model

import math
top_layers_file_path="/content/drive/My Drive/database_folder/car_theft/models/top_layers.iv3.hdf5"

checkpoint = ModelCheckpoint(top_layers_file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
tb = TensorBoard(log_dir='/content/drive/My Drive/database_folder/car_theft/models/./logs', batch_size=val_flow.batch_size, write_graph=True, update_freq='batch')
early = EarlyStopping(monitor="loss", mode="min", patience=5)
csv_logger = CSVLogger('/content/drive/My Drive/database_folder/car_theft/models/./logs/iv3-log.csv', append=True)

history = model.fit_generator(train_flow, 
                              epochs=5, 
                              verbose=1,
                              validation_data=val_flow,
                              validation_steps=math.ceil(val_flow.samples/val_flow.batch_size),
                              steps_per_epoch=math.ceil(train_flow.samples/train_flow.batch_size),
                              callbacks=[checkpoint, early, tb, csv_logger])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 54 could not be retrieved. It could be because a worker has died.
  UserWarning)


 1/76 [..............................] - ETA: 50:25 - loss: 2.3873 - acc: 0.0781 - top_k_categorical_accuracy: 0.5000WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1265: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.

 3/76 [>.............................] - ETA: 36:42 - loss: 3.5319 - acc: 0.1406 - top_k_categorical_accuracy: 0.5938

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 2 could not be retrieved. It could be because a worker has died.
  UserWarning)


20/76 [======>.......................] - ETA: 25:57 - loss: 2.5515 - acc: 0.2406 - top_k_categorical_accuracy: 0.7539

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 53 could not be retrieved. It could be because a worker has died.
  UserWarning)


76/76 [==============================] - 2031s 27s/step - loss: 1.7745 - acc: 0.3799 - top_k_categorical_accuracy: 0.8864 - val_loss: 1.6234 - val_acc: 0.3417 - val_top_k_categorical_accuracy: 0.9160

Epoch 00001: loss improved from inf to 1.77529, saving model to /content/drive/My Drive/database_folder/car_theft/models/top_layers.iv3.hdf5
Epoch 2/5


In [15]:
#evaluating model
import math
top_layers_file_path="/content/drive/My Drive/database_folder/car_theft/models/top_layers.iv3.hdf5"

model.load_weights(top_layers_file_path)
loss,acc,top_5=model.evaluate_generator(
    test_flow,
    verbose=True,
    steps=math.ceil(test_flow.samples/test_flow.batch_size)
)

print("loss: ",loss)
print("Acc: ",acc)
print("Top 5: ",top_5)


KeyboardInterrupt: ignored

In [0]:
label=[k for k,v in train_flow.class_indices.items()]
with open('/content/drive/My Drive/database_folder/car_theft/models/iv3-labels.txt', 'w+')as file:
  file.write("\n.join(label)")

In [18]:
# Test model with sample image

from keras.preprocessing import image
import numpy as np
import glob
import random

# file_list=glob.glob("/content/drive/My Drive/database_folder/car_theft/test")
img_path="/content/drive/My Drive/database_folder/car_theft/test/ford_f150_2006/2007 FORD F-150_00Y0Y_4822RaOsEul_600x450.jpg"
img_cat=os.path.split(os.path.dirname(img_path))[1]

print("image category :",img_cat)
img=image.load_img(img_path,target_size=(299,299))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)

preds=model.predict(x)
print("raw_prediction: ",preds)

top_x=3
top_args=preds[0].argsort()[-top_x:][::-1]
preds_label=[label[p] for p in top_args]
print("\nTop " + str(top_x) + " confidence: " + " ".join(map(str, sorted(preds[0])[-top_x:][::-1])))
print("Top " + str(top_x) + " labels: " + " ".join(map(str, preds_label)))

image category : ford_f150_2006
raw_prediction:  [[1.4968957e-04 4.2778128e-01 2.8079495e-01 1.9621055e-01 9.0591006e-02
  1.3205447e-03 2.2504255e-03 1.8846220e-04 4.0358890e-04 3.0952424e-04]]

Top 3 confidence: 0.42778128 0.28079495 0.19621055
Top 3 labels: chevrolet_silverado_2004 dodge_ram_2001 ford_f150_2006


transform keras model to tensorfllow frozen graph

In [28]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

# top_layers_file_path="/content/drive/My Drive/database_folder/car_theft/models/top_layers.iv3.hdf5"

input_model_path=top_layers_file_path
output_model_name="top_layers.iv3.pb"
ouput_model_dir="/content/drive/My Drive/database_folder/car_theft/models/tf_model"

K.set_learning_phase(0)
sess=K.get_session

test_model=models.load_model(input_model_path)
orig_output_node_names=[node.op.name for node in test_model.outputs]

constant_graph=graph_util.convert_variables_to_constants(
    sess,
    sess.graph.as_graph_def(),
    orig_output_node_names
)

graph_io.write_graph(
    constant_graph,
    output_model_dir,
    output_model_name,
    as_text=False
)

AttributeError: ignored

In [25]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

top_layers_file_path="/content/drive/My Drive/database_folder/car_theft/models/top_layers.iv3.hdf5"

input_model_path = top_layers_file_path

output_model_name="top_layers.iv3.pb"
ouput_model_dir="/content/drive/My Drive/database_folder/car_theft/models/tf_model"

output_model_dir = "tf_model"

K.set_learning_phase(0)
sess = K.get_session()

test_model = models.load_model(input_model_path)
orig_output_node_names = [node.op.name for node in test_model.outputs]

constant_graph = graph_util.convert_variables_to_constants(
    sess,
    sess.graph.as_graph_def(),
    orig_output_node_names)
graph_io.write_graph(
    constant_graph,
    output_model_dir,
    output_model_name,
    as_text=False)

INFO:tensorflow:Froze 380 variables.
INFO:tensorflow:Converted 380 variables to const ops.


'tf_model/top_layers.iv3.pb'